In [92]:
!pip install langchain==0.1.8
!pip install langchain-openai==0.0.6
!pip install python-dotenv==1.0.0
!pip install chromadb==0.4.18

DEPRECATION: dropbox 11.36.0 has a non-standard dependency specifier stone>=2.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of dropbox or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: dropbox 11.36.0 has a non-standard dependency specifier stone>=2.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of dropbox or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: dropbox 11.36.0 has a non-standard dependency specifier stone>=2.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of dropbox or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


   ---------------------------------------- 0.0/125.0 kB ? eta -:--:--
   ---------------------------------------  122.9/125.0 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------  122.9/125.0 kB 3.5 MB/s eta 0:00:01
   -------------------------------------- 125.0/125.0 kB 917.9 kB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 9.1
    Uninstalling websockets-9.1:
      Successfully uninstalled websockets-9.1


DEPRECATION: dropbox 11.36.0 has a non-standard dependency specifier stone>=2.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of dropbox or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
realtime 1.0.0 requires websockets<11.0,>=10.3, but you have websockets 12.0 which is incompatible.
realtime-py 0.1.3 requires websockets<10.0,>=9.1, but you have websockets 12.0 which is incompatible.
supabase 1.0.3 requires httpx<0.24.0,>=0.23.0, but you have httpx 0.27.0 which is incompatible.
supabase-py 0.0.2 requires gotrue==0.2.0, but you have gotrue 1.3.1 which is incompatible.
supabase-py 0.0.2 requires requests==2.25.1, but you have requests 2.31.0

In [73]:
import os
import json
import yaml
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [3]:
load_dotenv()

True

In [111]:
llm = ChatOpenAI(model="gpt-4-turbo-preview",
                 temperature=0.7,
                 model_kwargs={
                    "frequency_penalty": 0.0,
                     "presence_penalty": 0.0,
                     "top_p": 1.0,
                  })

In [34]:
def write_to_file(filename, text):
    try:
        directory = os.path.dirname(filename)
        if not os.path.exists(directory):
            os.makedirs(directory)
        with open(filename, 'a') as file:
            file.write(text)
        print("Text successfully written to", filename)
    except Exception as e:
        print("An error occurred:", str(e))

In [81]:
def save_dict_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

In [5]:
first_cycle_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", """
    Read the following transcript and extract all the arguments made about AI safety. Make sure they are self-contained.

  You must stick as close as possible to the transcript - use the author's own words and tone of voice.

  You must write each argument in a valid YAML format, surrounded with backticks.
  You must separate each argument with a new line.


  The simplest possible argument must at least contain three claims:

  ```yaml
  claim: "Top-level claim"
  premises:
    - claim: "First independent premise supporting the top-level claim"
    - claim: "Second independent premise supporting the top-level claim"
  ```

  And here's an example of a more complex argument, which also includes examples to illustrate lower-level claims:

  ```yaml
  claim: "Top-level claim"
  premises:
    - claim: "First independent premise supporting the top-level claim"
    - claim: "Supporting premise for the top-level claim"
      example: "Example supporting this premise"
    - claim: "Another supporting premise for the top-level claim"
    - claim: "Second independent premise supporting the top-level claim"
      premises:
        - claim: "Supporting premise for the second independent premise"
        - claim: "Another supporting premise for the second independent premise"
        - claim: "Independent premise supporting the second independent premise"
          example: "Example supporting this independent premise"
  ```

  Here's how to read this structure:

  The top-level claim is the main argument.
  Directly nested under the claim are independent premises. These provide justification independently of other premises.
  An example can be nested directly under a claim to provide further context or support.
  Just like the top-level claim, each premise can itself be supported by further individual premises, or examples, creating a nested structure.

  # Here is the transcript:

  {transcript}
  """)
])

In [6]:
second_cycle_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", """
  Based on the following transcript, make the arguments clear and distinct. 
  You may need to merge similar arguments to create a better, more logical argument.

  Create the best, strongest possible version of the arguments, here's what to do:

  - Make sure the arguments is self-contained
  - Make the arguments understandable on their own, out-of-context
  - Remember, arguments are not a description or an explanation
  - Premise must always give a reason to believe the claim above
  - Avoid using pronouns in premises
  - A claim can have a maximum of two child claims (premises), rewrite if needed

  # Argument format 

  You must write each argument in valid YAML format, surrounded with backticks.
  Separate each argument with new line.
  You must stick as closely as possible to the transcript. 
  Above all, you must express the argument in the words of the author, stick as close as possible to the tone of voice and phrases used in the transcript.



  Here are some examples:

  A simple argument might look like this: 

  ```yaml
  claim: "Top-level claim"
  premises:
    - claim: "First independent premise supporting the top-level claim"
    - claim: "Second independent premise supporting the top-level claim"
  ```

  And here's an example of a more complex argument, which also includes examples to illustrate lower-level claims:

  ```yaml
  claim: "Top-level claim"
  premises:
    - claim: "First independent premise supporting the top-level claim"
    - claim: "Supporting premise for the top-level claim"
      example: "Example supporting this premise"
    - claim: "Another supporting premise for the top-level claim"
    - claim: "Second independent premise supporting the top-level claim"
      premises:
        - claim: "Supporting premise for the second independent premise"
        - claim: "Another supporting premise for the second independent premise"
        - claim: "Independent premise supporting the second independent premise"
          example: "Example supporting this independent premise"
  ```

  Here's how to read this structure:

  The top-level claim is the main argument.
  Directly nested under the claim are independent premises. These provide justification independently of other premises.
  An example can be nested directly under a claim to provide further context or support.
  Just like the top-level claim, each premise can itself be supported by further individual premises, or examples, creating a nested structure.


  # Arguments to improve:

  {all_arguments}

  # Transcript

  {transcript}
  """)
])

In [110]:
third_cycle_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world expert at creating accessible, persuasive explanations."),
    ("user", """
  Based on your own knowledge and the transcript, create a structured explanation for the following argument. Use the context from the transcript for the explanation.

  # Argument to use for the explanation

  {argument}


  The structured explanation must be directly based on the argument. You can also use the provided transcript for context.

  You must follow this YAML format:

  ```yaml
  counteragument_to: (what would be the argument, to which this argument is a counterargument? use your own knowledge. use bullet points)

  strongest_objection: (what is the strongest, good-faith, honest objection that a thoughful person might have? use bullet points)
  consequences_if_true: (if true, what would be the consequences? write in causal language,  use bullet points, max 3)

  link_to_ai_safety: (how is this linked to AI safety? 1 sentence.)

  simple_explanation: (explain this clearly to a college student in max. 4 sentences, speak persuasively as the author of this argument. don't use bullet points)

  examples: (max 3 examples, use bullet points)

  ```

  # Here is the transcript:

  {transcript}
  """)
])

In [70]:
def load_text_from_sources_and_make_chunks(directory):
    folder_names = []
    raw_texts = []
    for entry in os.listdir(directory):
        folder_names.append(entry)
        print(folder_names)
    for folder_name in folder_names:
        loader = DirectoryLoader(f'./sources/{folder_name}/source', glob="**/*.md")
        text_splitter = CharacterTextSplitter(chunk_size=10000,
        chunk_overlap=0,
        length_function=len)
        raw_text = loader.load()
        entire_text_from_a_source = ""
        for doc in raw_text:
            entire_text_from_a_source += doc.page_content
        chunks = text_splitter.create_documents([entire_text_from_a_source])
        formatted_chunks = []
        for chunk in chunks:
            formatted_chunks.append(chunk.page_content)
        temp = {
            "name" : folder_name,
            "path": f'./sources/{folder_name}',
            "chunks": formatted_chunks,
        }
        raw_texts.append(temp)
    return raw_texts

In [71]:
sources_dicts = load_text_from_sources_and_make_chunks("./sources")

['connor-feb24-001']
['connor-feb24-001', 'daniel-j-feb24-001']
['connor-feb24-001', 'daniel-j-feb24-001', 'eliezer-feb24-001']


Created a chunk of size 10030, which is longer than the specified 10000


In [73]:
def first_cycle_of_extracting_arguments(dicts):
    first_cycle_chain = first_cycle_prompt | llm
    for dict in dicts:
        dict["arguments"] = []
        for chunk in dict["chunks"]:
            first_cycle_response = first_cycle_chain.invoke({"transcript": chunk})
            dict["arguments"].append(first_cycle_response.content)
        print("over")
        text = '\n\n'.join(dict["arguments"])
        filename = f"{dict['path']}/steps/first_step.md"
        write_to_file(filename, text)
    return dicts

In [74]:
dicts_with_extracted_args = first_cycle_of_extracting_arguments(sources_dicts)

over
Text successfully written to ./sources/connor-feb24-001/steps/first_step.md
over
Text successfully written to ./sources/daniel-j-feb24-001/steps/first_step.md
over
Text successfully written to ./sources/eliezer-feb24-001/steps/first_step.md


In [84]:
save_dict_to_json(dicts_with_extracted_args, "./sources/first_cycle_results.json")

In [86]:
def second_cycle_of_extracting_arguments(dicts):
    second_cycle_chain = second_cycle_prompt | llm
    for dict in dicts:
        dict["improved_arguments"] = []
        for i, chunk in enumerate(dict["chunks"]):
            second_cycle_response = second_cycle_chain.invoke({"all_arguments": dict["arguments"][i], "transcript": chunk, })
            dict["improved_arguments"].append(second_cycle_response.content)
        print("over")
        text = '\n\n'.join(dict["improved_arguments"])
        filename = f"{dict['path']}/steps/second_step.md"
        write_to_file(filename, text)
    return dicts

In [87]:
dicts_with_extracted_and_improved_args = second_cycle_of_extracting_arguments(dicts_with_extracted_args)

over
Text successfully written to ./sources/connor-feb24-001/steps/second_step.md
over
Text successfully written to ./sources/daniel-j-feb24-001/steps/second_step.md
over
Text successfully written to ./sources/eliezer-feb24-001/steps/second_step.md


In [101]:
def split_improved_arguments_in_dict(dicts):
    for dict in dicts:
        dict["isolated_arguments"] = []
        for improved_arg in dict["improved_arguments"]:
            if improved_arg:
                splitted_args = improved_arg.split("```yaml")
                splitted_args_cleaned = []
                for arg in splitted_args:
                    arg_clean = arg.split("```")[0]
                    if (arg_clean != ""):
                        splitted_args_cleaned.append(arg_clean.strip())   
                if splitted_args_cleaned != None:
                    dict["isolated_arguments"].append(splitted_args_cleaned)
    return dicts

In [102]:
dicts_with_isolated_improved_arguments = split_improved_arguments_in_dict(dicts_with_extracted_and_improved_args)

In [90]:
save_dict_to_json(dicts_with_isolated_improved_arguments, "./sources/second_cycle_results.json")

In [107]:
dicts_with_isolated_improved_arguments = load_results_from_json(2)

In [88]:
def create_embeddings_with_smaller_chunks(directory):
    folder_names = []
    all_chunks = []
    dbs = []
    for entry in os.listdir(directory):
        if entry != "json":
            folder_names.append(entry)
    for folder_name in folder_names:
        loader = DirectoryLoader(f'./sources/{folder_name}/source', glob="**/*.md")
        text_splitter = CharacterTextSplitter(chunk_size=1000,
        chunk_overlap=100,
        length_function=len)
        raw_text = loader.load()
        entire_text_from_a_source = ""
        for doc in raw_text:
            entire_text_from_a_source += doc.page_content
        chunks = text_splitter.create_documents([entire_text_from_a_source])
        for chunk in chunks:
            chunk.metadata = {"folder": folder_name}
            all_chunks.append(chunk)
        embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
        db = Chroma.from_documents(all_chunks, embeddings)
        dbs.append(db)
    return dbs

In [89]:
dbs = create_embeddings_with_smaller_chunks("./sources")

Created a chunk of size 9533, which is longer than the specified 1000
Created a chunk of size 10030, which is longer than the specified 1000
Created a chunk of size 8684, which is longer than the specified 1000
Created a chunk of size 9803, which is longer than the specified 1000
Created a chunk of size 9280, which is longer than the specified 1000
Created a chunk of size 9937, which is longer than the specified 1000
Created a chunk of size 3719, which is longer than the specified 1000
Created a chunk of size 9548, which is longer than the specified 1000
Created a chunk of size 9798, which is longer than the specified 1000
Created a chunk of size 7692, which is longer than the specified 1000
Created a chunk of size 7906, which is longer than the specified 1000
Created a chunk of size 8458, which is longer than the specified 1000
Created a chunk of size 2107, which is longer than the specified 1000
Created a chunk of size 3188, which is longer than the specified 1000
Created a chunk of 

In [103]:
def third_cycle_of_extracting_arguments(dicts):
    third_cycle_chain = third_cycle_prompt | llm
    for i, dict in enumerate(dicts):
        dict["explanations"] = []
        db = dbs[i]
        for i, chunk in enumerate(dict["chunks"]):
            explanations = []
            for arg in dict["isolated_arguments"][i]:
                docs = db.similarity_search(arg, k=3)
                print(docs)
                context = ""
                for doc in docs:
                    context += "\n" + doc.page_content
                explanation = third_cycle_chain.invoke({"argument": arg, "transcript": context})
                explanations.append(explanation.content.split("```yaml\n")[1].split("```")[0].strip())
            dict["explanations"].append(explanations)
    return dicts

In [112]:
dicts_with_arguments_and_explanations = third_cycle_of_extracting_arguments(dicts_with_isolated_improved_arguments)

[Document(page_content='Speaker 2\nAnd so, as we stand at the dawn of a new age of unprecedented AI progress, it is tempting to think of this topic as just a novel fad of eccentric of AGI, rather than a fundamental question that has been with us since the very beginning of the field of computer science. The definition of existential risk is a risk that threatens the premature extinction of humanity, or the permanent and drastic destruction of its potential for desirable future development, the loss of control of our future. And if we build machines that are more intelligent, more competent in people, that are more capable at manipulation, deception, politics, science, business, and everything else, and we do not control them, then the future will belong to them, not to us. Everything else derives from this simple observation.', metadata={'folder': 'connor-feb24-001'}), Document(page_content='Speaker 2\nAnd so, as we stand at the dawn of a new age of unprecedented AI progress, it is tem


KeyboardInterrupt



In [122]:
print(dicts_with_arguments_and_explanations)

[{'name': 'connor-feb24-001', 'path': './sources/connor-feb24-001', 'chunks': ["Speaker 1\nGood evening, everyone. Sorry we're a little late. So tonight we debate AI, the motion that this house believes artificial intelligence is an existential threat. And we're honored tonight to be joined by a particular distinguished individuals. In proposition, we have Connolly, Rebecca Gorman and Professor Joanna Bryson. While in opposition, Professor Judy Weisman, Dr. Indra Joshi are joined by student Speaker Igor Sterner from Pembroke College. Before we begin, let's just recap the format for the evening. So each paper speaker tonight has ten minutes to make their case. The first and last minutes are protected time, so please do not try and poi them during that time. But as members, this is your house and welcome contributions from all. So between each round of paper speakers, there will be time for speeches from the floor in proposition opposition and abstention.\n\nSpeaker 1\nShould you wish to

In [123]:
save_dict_to_json(dicts_with_arguments_and_explanations, "./sources/third_cycle_results.json")

In [1]:
print(len(dicts_with_arguments_and_explanations[2]["isolated_arguments"][51]))

NameError: name 'dicts_with_arguments_and_explanations' is not defined

In [113]:
def load_results_from_json(int):
    if int == 1:
        with open("./sources/json/first_cycle_results.json", 'r') as file:
            data = json.load(file)
            return data
    if int == 2:
        with open("./sources/json/second_cycle_results.json", 'r') as file:
            data = json.load(file)
            return data
    if int == 3:
        with open("./sources/json/third_cycle_results.json", 'r') as file:
            data = json.load(file)
            return data

In [7]:
dicts_with_arguments_and_explanations = load_results_from_json(3)

In [36]:
def save_third_step(dicts):
    for dict in dicts:
        dict["final_arguments"] = []
        for i, isolated_argument_group in enumerate(dict["isolated_arguments"]):
            for j, single_isolated_arg in enumerate(isolated_argument_group):
                print(i)
                print(j)
                final_arg = "\n".join([single_isolated_arg, dict["explanations"][i][j]])
                dict["final_arguments"].append(final_arg)
                final_arg_yaml_format = " ```yaml\n" + final_arg + "\n```\n\n"
                filename = f"{dict['path']}/steps/third_step.md"
                write_to_file(filename, final_arg_yaml_format)
    return dicts

In [38]:
final_dicts = save_third_step(dicts_with_arguments_and_explanations)

0
0
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
0
1
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
0
2
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
0
3
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
0
4
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
1
0
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
1
1
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
1
2
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
1
3
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
1
4
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
1
5
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
2
0
Text successfully written to ./sources/connor-feb24-001/steps/third_step.md
2
1
Text successfully written to ./sourc

In [60]:
final_dicts[0]["final_arguments"]

['claim: "Artificial intelligence is an existential threat."\npremises:\n  - claim: "Building more intelligent and competent machines without control leads to a future dominated by them, not humans."\n  - claim: "The goal of major companies is to develop fully autonomous AGI agents, while our ability to control and understand these agents is minimal."\n  - claim: "The existential threat stems from the extraordinary competence and autonomy of machines that are poorly controlled."\ncounterargument_to:\n  - Artificial intelligence will enhance human capabilities and is not a threat to humanity.\n  - AI development is inherently safe and controllable.\n\nstrongest_objjection:\n  - Advanced AI systems could be designed with inherent safety measures and ethical constraints, significantly reducing the risk of them becoming uncontrollable or posing an existential threat.\n\nconsequences_if_true:\n  - The future could be dominated by machines rather than humans, leading to the potential extinct

In [80]:
def create_directory_structure_for_chatbots(directory_name):
    script_directory = os.getcwd()
    new_directory_path = os.path.join(script_directory, directory_name)
    if not os.path.exists(new_directory_path):
        os.makedirs(new_directory_path)
        yaml_file_path = os.path.join(new_directory_path, "metadata.yaml")
        with open(yaml_file_path, "w") as yaml_file:
            with open(yaml_file_path, "w") as yaml_file:
                yaml.dump({"name": "", "tags": None, "based_on": ""}, yaml_file)

        prompts_folder_path = os.path.join(new_directory_path, "prompts")
        knowledge_base_folder_path = os.path.join(new_directory_path, "knowledge_base")
        for folder_path in [prompts_folder_path, knowledge_base_folder_path]:
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
                system_prompt_file_path = os.path.join(prompts_folder_path, "system_prompt.md")
                with open(system_prompt_file_path, "w") as system_prompt_file:
                    system_prompt_file.write("System Prompt")
    if os.path.exists(new_directory_path):
        yaml_file_path = os.path.join(new_directory_path, "metadata.yaml")
        with open(yaml_file_path, "w") as yaml_file:
            yaml.dump({"name": "", "tags": None, "based_on": ""}, yaml_file)
        prompts_folder_path = os.path.join(new_directory_path, "prompts")
        knowledge_base_folder_path = os.path.join(new_directory_path, "knowledge_base")
        for folder_path in [prompts_folder_path, knowledge_base_folder_path]:
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

In [83]:
def create_chatbots(dicts):
    for dict in dicts:
        create_directory_structure_for_chatbots(dict["name"])
        for i, final_arg in enumerate(dict["final_arguments"]):
            filename = f'./{dict["name"]}/knowledge_base/{dict["name"]}-{str(i + 1)}.md'
            with open(filename, 'w') as file:
                file.write(final_arg)
            prompt_filename = f'./{dict["name"]}/prompts/system_prompt.md'
            with open(prompt_filename, 'a') as file:
                file.write("Write system prompt here")

In [84]:
create_chatbots(final_dicts)